In [1]:
using CSV, DataFrames, Statistics, CategoricalArrays

In [4]:
train_path = "/Users/gabemiller/Desktop/MIT/Edge/project/train.csv"
test_path = "/Users/gabemiller/Desktop/MIT/Edge/project/test.csv"

train = CSV.read(train_path, DataFrame)
test = CSV.read(test_path, DataFrame)

println("Columns: ", names(train))
println("First 5 rows: ", first(train, 5))

Columns: ["PeopleId", "activityID", "heart_rate_mean", "heart_rate_std", "hand temperature (°C)_mean", "hand temperature (°C)_std", "hand acceleration X ±16g_mean", "hand acceleration X ±16g_std", "hand acceleration Y ±16g_mean", "hand acceleration Y ±16g_std", "hand acceleration Z ±16g_mean", "hand acceleration Z ±16g_std", "hand gyroscope X_mean", "hand gyroscope X_std", "hand gyroscope Y_mean", "hand gyroscope Y_std", "hand gyroscope Z_mean", "hand gyroscope Z_std", "hand magnetometer X_mean", "hand magnetometer X_std", "hand magnetometer Y_mean", "hand magnetometer Y_std", "hand magnetometer Z_mean", "hand magnetometer Z_std", "chest temperature (°C)_mean", "chest temperature (°C)_std", "chest acceleration X ±16g_mean", "chest acceleration X ±16g_std", "chest acceleration Y ±16g_mean", "chest acceleration Y ±16g_std", "chest acceleration Z ±16g_mean", "chest acceleration Z ±16g_std", "chest gyroscope X_mean", "chest gyroscope X_std", "chest gyroscope Y_mean", "chest gyroscope Y_std

In [3]:
# Downsample data to 20Hz
function downsample_data(df, factor)
    return df[1:factor:end, :]
end

# Sort primarily by PeopleId and secondarily by activityID]
sort!(train, [:PeopleId, :activityID])
sort!(test, [:PeopleId, :activityID])

# Apply downsampling
train = downsample_data(train, 10)  # Downsampling 100Hz to 20Hz
test = downsample_data(test, 10)

Row,Column1,activityID,heart_rate,hand temperature (°C),hand acceleration X ±16g,hand acceleration Y ±16g,hand acceleration Z ±16g,hand gyroscope X,hand gyroscope Y,hand gyroscope Z,hand magnetometer X,hand magnetometer Y,hand magnetometer Z,chest temperature (°C),chest acceleration X ±16g,chest acceleration Y ±16g,chest acceleration Z ±16g,chest gyroscope X,chest gyroscope Y,chest gyroscope Z,chest magnetometer X,chest magnetometer Y,chest magnetometer Z,ankle temperature (°C),ankle acceleration X ±16g,ankle acceleration Y ±16g,ankle acceleration Z ±16g,ankle gyroscope X,ankle gyroscope Y,ankle gyroscope Z,ankle magnetometer X,ankle magnetometer Y,ankle magnetometer Z,PeopleId
,Int64,String31,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,2767460,Nordic walking,114.0,30.6875,-1.1347,-10.5446,2.17547,-0.157993,0.177149,-0.127296,4.20641,20.8685,-28.8394,34.0,0.289925,9.67001,1.37135,0.0908165,0.137308,-0.0229764,-3.96984,-24.5603,-8.07506,31.8125,9.68533,-0.00680776,-2.11642,-0.0450717,-0.0111798,0.0183236,-24.2099,-10.8712,16.3699,8
2,2767470,Nordic walking,114.0,30.6875,1.25183,-10.0594,3.36421,0.260787,-0.119337,-1.338,2.35991,20.2218,-29.8816,34.0,0.814232,10.0425,1.03115,0.125612,-0.110944,-0.0270999,-3.83783,-25.0813,-7.78918,31.8125,9.64449,0.184557,-2.27238,-0.0239491,-0.013904,-0.0115804,-24.0935,-10.4916,16.0862,8
3,2767480,Nordic walking,113.545,30.6875,1.43048,-7.66776,3.66671,0.839185,0.147904,-0.907363,-0.346267,18.2565,-29.9954,34.0,0.430248,9.40337,0.987551,0.0228744,-0.063328,0.065003,-4.74311,-24.6609,-7.93887,31.8125,9.79749,0.182645,-2.15503,-0.00731106,-0.0309869,0.0387184,-24.2006,-11.3871,16.2287,8
4,2767490,Nordic walking,113.0,30.6875,1.61993,-5.69973,3.27804,0.0742076,0.25452,0.0712577,-0.201417,17.6273,-31.4354,34.0,0.507274,9.70435,0.987968,0.0494763,0.0243738,-0.000823731,-4.73831,-25.4353,-7.51142,31.8125,9.83331,0.296956,-2.23256,-0.0382837,-0.0353787,-0.0157583,-24.849,-10.9689,16.0841,8
5,2767500,Nordic walking,113.0,30.6875,0.492024,-5.93631,1.29909,-0.660706,0.531046,1.15603,1.51231,18.9016,-30.3968,34.0,0.436465,9.59293,1.18065,0.171719,-0.04701,-0.0569582,-5.00184,-25.0386,-7.51551,31.8125,9.56893,0.26129,-2.31222,0.0312191,-0.0467679,-0.0125312,-24.7349,-10.3309,16.0838,8
6,2767510,Nordic walking,113.0,30.6875,1.17163,-100.679,22.9894,-3.10991,0.364184,0.560474,7.92547,21.2907,-28.4558,34.0,0.81035,9.43957,0.955599,0.102688,-0.0829663,0.0357981,-4.09816,-25.201,-7.5083,31.8125,9.60246,0.148111,-2.42695,0.0110715,-0.0110933,-0.0114619,-24.6044,-10.4662,16.0844,8
7,2767520,Nordic walking,113.0,30.625,-1.38352,-6.97865,1.89031,0.95019,0.681064,-0.121623,7.18912,21.5771,-30.0308,33.9375,1.0682,9.51197,0.650267,0.0822691,-0.0727393,-0.0170194,-4.10332,-24.5556,-7.79401,31.8125,9.60797,0.146504,-2.23393,0.0179317,-0.0118802,0.0121809,-24.4708,-11.1186,15.3765,8
8,2767530,Nordic walking,113.0,30.625,-0.848875,-9.9256,3.75565,0.0478293,0.315445,-0.2764,7.20105,20.212,-29.2351,33.9375,0.648594,9.81425,0.602962,-0.00869718,-0.0314441,-0.10701,-5.27615,-25.1587,-6.3887,31.8125,9.60746,0.260732,-2.27324,0.00204032,0.0504242,0.0103602,-23.963,-10.6269,16.0868,8
9,2767540,Nordic walking,113.0,30.625,-1.34337,-9.05155,3.47588,-0.0574203,0.25694,-0.178556,7.45123,20.8183,-30.4204,33.9375,0.54902,9.78032,1.10464,0.0176968,-0.0339971,-0.0661225,-5.02368,-24.5229,-6.67256,31.8125,9.67952,0.146994,-2.34897,0.0131188,0.0493599,0.0229832,-24.4743,-10.731,15.8015,8


In [5]:
# List of columns to exclude from normalization
exclude_cols = ["activityID", "PeopleId"]  # Columns to exclude from normalization

2-element Vector{String}:
 "activityID"
 "PeopleId"

In [7]:
# Handle missing values (if any)
train = dropmissing(train)
test = dropmissing(test);

In [8]:
# Convert CategoricalValue to strings
y_train = DataFrame(class = train.activityID)  # Create a DataFrame with one column named `task1`
y_test = DataFrame(class = test.activityID)

# Prepare feature matrices
X_train = Matrix(select(train, Not([:activityID, :PeopleId])))
X_test = Matrix(select(test, Not([:activityID, :PeopleId])))

# Print train_df.activityID datatype
println("train_df.activityID datatype: ", typeof(train.activityID))

train_df.activityID datatype: PooledArrays.PooledVector{String31, UInt32, Vector{UInt32}}


In [9]:
# Train an OCT (this make take some time, ~3-5 minutes)

grid_oct1 = IAI.GridSearch(
    IAI.OptimalTreeMultiClassifier(),
    max_depth = 2:6,                                    # Depths from 2 to 6
    minbucket = [5, 10, 20, 30, 40, 50],                # Minbucket values
    max_categoric_levels_before_warning = 20            # Get rid of warnings
)

IAI.fit!(grid_oct1, X_train, y_train)

┌ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
└ @ nothing nothing:nothing
┌ Warning: Interpretable AI license expires soon: 2024-12-31T00:00:00. If you need to renew, please send us the following machine ID:
│ b1de414b5a9acbdef4d34422ebcfa7b9d570b193c59ff39c1f36173200a1f28f
└ @ nothing nothing:nothing


InterruptException: InterruptException:

In [14]:
# Get best model
lnr1 = IAI.get_learner(grid_oct)

# Display selected model's parameters
params = IAI.get_params(lnr1)
println("Max depth of the best model is ", params[:max_depth])
println("Minbucket size of the best model is ", params[:minbucket])
println("Criterion of the best model is ", params[:criterion])

In [15]:
# Calculate score for training set (note: for the full training set)
train1_score = IAI.score(lnr1, X_train, Array(y_train))

println("Training score: ", round(train1_score, digits=4))

UndefVarError: UndefVarError: `X_train` not defined

In [16]:
# Calculate AUC score for test data
test1_score = IAI.score(lnr1, X_test, Array(y_test))

println("Test score: ", round(test1_score, digits=4))

UndefVarError: UndefVarError: `X_test` not defined